## Dask demo notebook

This notebook shows a demonstration of how to use dask and CMAC2.0 on a JupyterHub node.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import dask.bag as db
import pyart
import importlib
import netCDF4
import re
import os
import subprocess
import sys
import imageio
import numpy as np

from glob import glob
from cmac import cmac, quicklooks, get_sounding_times, get_sounding_file_name, config
from IPython.display import Image, display
from dask_jobqueue import SLURMCluster, PBSCluster
from datetime import datetime
from distributed import Client
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client, metrics, wait
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, progress

This subroutine, will do both the processing and plotting for one radar file. A more sophisicated version of this is contained within the cmac_dask script.

In [ ]:
def run_cmac_and_plotting(radar_file_path, cmac_config, sonde_times,
                          sounding_files, clutter_file_path, geotiff,
                          out_path, img_directory, sweep=3, dd_lobes=False):
    """ For dask we need the radar plotting routines all in one subroutine. """
    match_datetime = re.search(r'\d{4}\d{2}\d{2}.\d{6}', radar_file_path)
    match_month = re.search(r'\d{4}\d{2}', radar_file_path)
    file_datetime = match_datetime.group()
    file_month = match_month.group()
    save_name = cmac_config['save_name']
    file_name = (out_path + file_month + '/' + save_name + '.'
                 + file_datetime + '.nc')
    if os.path.exists(file_name):
        import gc
        gc.collect()
        return
    
    try:
        radar = pyart.io.read(radar_file_path)
    except TypeError:
        print(radar_file_path + ' has encountered TypeError!')
        return

    
    #radar_start_date = netCDF4.num2date(radar.time['data'][0],
     #                                   radar.time['units'], 
      #                                  only_use_cftime_datetimes=False, only_use_python_datetimes=True)

    #year_str = "%04d" % radar_start_date.year
    #month_str = "%02d" % radar_start_date.month
    #day_str = "%02d" % radar_start_date.day
    #hour_str = "%02d" % radar_start_date.hour
    #minute_str = "%02d" % radar_start_date.minute
    #second_str = "%02d" % radar_start_date.second
    #save_name = cmac_config['save_name']
    #file_name = (out_path + year_str + month_str + '/' + save_name + '.'
     #            + year_str + month_str + day_str + '.' + hour_str
      #           + minute_str + second_str + '.nc')
    rad_time = datetime.strptime(radar.time["units"][0:33], "seconds since %Y-%m-%d %H:%M:%S")
    # Load clutter files.
    #clutter = pyart.io.read(
     #   clutter_file_path+'clutter_corcsapr2cfrppiM1.a1'
      #  + '.' + year_str + month_str + day_str + '.' + hour_str
       # + minute_str + second_str + '.nc')
    #clutter_field_dict = clutter.fields['ground_clutter']
    #radar.add_field(
     #   'ground_clutter', clutter_field_dict, replace_existing=True)
    #del clutter
    sonde_index = np.argmin(np.abs(sonde_times - rad_time))
    sounding_file = sounding_files[sonde_index]
    # Retrieve closest sonde in time to the time of the radar file.
    sonde = netCDF4.Dataset(sounding_file)
    # Running the cmac code to produce a cmac_radar object.
    try:
        cmac_radar = cmac(radar, sonde, 'sail_xband_ppi', geotiff=geotiff)
    except ValueError:
        del radar
        sonde.close()
        import gc
        gc.collect()
        return
    # Free up some memory.
    del radar
    sonde.close()

    # Produce the cmac_radar file from the cmac_radar object.
    pyart.io.write_cfradial(file_name, cmac_radar)
    print('## A CMAC radar object has been created at ' + file_name)

    if not os.path.exists(img_directory):
        os.makedirs(img_directory)
        subprocess.call('chmod -R g+rw ' + img_directory, shell=True)
    img_directory = img_directory + file_month
    # Producing all the cmac_radar quicklooks.
    quicklooks(cmac_radar, 'cacti_csapr2_ppi',
        dd_lobes=False, image_directory=img_directory)

    # Delete the cmac_radar object and move on to the next radar file.
    del cmac_radar
    import gc
    gc.collect()
    return

In [ ]:
import os

In [ ]:
os.path.exists

We have to start our dask cluster. Normally, the script to do this on stratus is in qsub_xsapr. The dask-scheduler has to be running on one compute node and the dask-workers on the other computer nodes. However, since we are only on one node, we can just use multiprocessing to do our work for us.

CMAC2.0 has different dictionaries that specify various configurations for the 3 XSAPRs in SGP. Here we will load i5 since the demo data is from XSAPR i5.

In [ ]:
meta_config = config.get_metadata('cacti_csapr2_ppi')
cmac_config = config.get_cmac_values('cacti_csapr2_ppi')
field_config = config.get_field_names('cacti_csapr2_ppi')

We can then use a dask bag to map the radar list into distributed memory and then execute the processing code on each file using .map().compute() on the bag

In [ ]:
soundings_directory = '/lustre/or-hydra/cades-arm/proj-shared/corsondewnpnM1.b1/'
radar_directory = '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/*/*'
clutter_directory = '/lustre/or-hydra/cades-arm/proj-shared/csapr2_clutter/'
#radar_files = glob(radar_directory + '*/*', recursive=True)
radar_files = glob(radar_directory, recursive=True)
sounding_files = glob(soundings_directory + '*')
#clutter_files = glob(clutter_directory)
geotiff = '/lustre/or-hydra/cades-arm/proj-shared/cacti_radardomain.tiff'

img_directory = '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1.png/'
out_path = '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/'
log_path = '/lustre/or-hydra/cades-arm/proj-shared/csapr_log/'
#sonde_file = '/home/rjackson/i5_test_data/sgpsondewnpnC1.b1.20170926.113600.cdf'
#clutter_file_path = '/home/rjackson/clutter201709.nc'

In [ ]:
radar_files

In [ ]:
radar_files.sort()
sounding_files.sort()
#clutter_files.sort()

In [ ]:
def parse_sonde_times(file_list):
    time_list = []
    for name in file_list:
        time_list.append(datetime.strptime(name.split('/')[-1], 'corsondewnpnM1.b1.%Y%m%d.%H%M%S.cdf'))
    return np.array(time_list)
        
#def parse_clutter_times(file_list):
 #   time_list = []
  #  for name in file_list:
   #     time_list.append(datetime.strptime(name.split('/')[-1], 'clutter_corcsapr2cfrppiM1.a1.%Y%m%d.%H%M%S.nc'))
    #return np.array(time_list)

sonde_times = parse_sonde_times(sounding_files)
#clutter_times = parse_clutter_times(clutter_files)

In [ ]:
#clutter_times.sort()
sonde_times.sort()

In [ ]:
len(radar_files)

In [ ]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster1 = PBSCluster(processes=6, cores=36, walltime='4:30:00', memory='160GB',
                      name='dask-worker', interface='ib0', queue='arm_high_mem', project='arm',
                      job_extra=['-W group_list=cades-arm'], extra=['--no-dashboard'])
cluster1.scale(4*6)         # Ask for tn workers
client1 = Client(cluster1)  # Connect this local process to remote workers

In [ ]:
#client1.wait_for_workers()

In [ ]:
cluster1

In [ ]:
client1

In [ ]:
the_bag = db.from_sequence(radar_files)
the_function = lambda x: run_cmac_and_plotting(x, cmac_config, sonde_times,
                      sounding_files, clutter_directory, geotiff,
                      out_path, img_directory, sweep=3, dd_lobes=False)

futures1 = the_bag.map(the_function)

In [ ]:
futures1.compute()

In [15]:
!qselect -u zsherman | xargs qdel

In [ ]:
run_cmac_and_plotting(radar_files[9000], cmac_config, sonde_times,
                      sounding_files, clutter_directory, geotiff,
                      out_path, img_directory, sweep=3, dd_lobes=False)

In [ ]:
rad_file = radar_files[540]
radar = pyart.io.read(rad_file)

In [ ]:
rad_time = datetime.strptime(radar.time["units"][0:33], "seconds since %Y-%m-%d %H:%M:%S")
sonde_index = np.argmin(np.abs(sonde_times - rad_time))
sounding_file = sounding_files[sonde_index]
clutter_index = np.argmin(np.abs(clutter_times - rad_time))
clutter_file = clutter_files[clutter_index]

In [ ]:
clutter = pyart.io.read(clutter_file)
clutter_field_dict = clutter.fields['ground_clutter']

In [ ]:
clutter_field_dict['data'].shape

In [ ]:
for key in radar.fields.keys():
    print(radar.fields[key]['data'].shape)

In [ ]:
im_files = glob('/lcrc/group/earthscience/icrisologo/csapr_imgs/masked_corrected_reflectivity*.png')
im_files.sort()
images = []
for filename in im_files:
    images.append(imageio.imread(filename))
imageio.mimsave('refl_animation.gif', images, duration=0.5)

with open('refl_animation.gif','rb') as f:
    display(Image(f.read()))

In [ ]:
radar = pyart.io.read(
    '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cfrhsrhi.M1.a1/201901/corcsapr2cfrhsrhiauxM1.a0.20190125.211100.nc')

In [ ]:
radar.fields.keys()

In [ ]:
import glob

In [ ]:
files = glob.glob('/lustre/or-hydra/cades-arm/proj-shared/corsondewnpnM1.b1/*')

In [ ]:
files

In [ ]:
len(files)

In [ ]:
import netCDF4
import shutil
for file in files:
    data = netCDF4.Dataset(file)
    if data.variables['alt'][:].max() < 20000.0:
        shutil.move(file, '/lustre/or-hydra/cades-arm/proj-shared/bad_cor_sonde')

        data.close()
    else:
        data.close()

In [ ]:
data = netCDF4.Dataset(files[0])

In [ ]:
data

In [ ]:
for file in files:
    

In [ ]:
radar = pyart.io.read('/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/201810/cacticsapr2cmacppi.c1.20181031.091504.nc')

In [ ]:
radar.fields.keys()

In [ ]:
del radar

In [ ]:
radar.latitude

In [ ]:
radar.longitude

In [ ]:
import re
from datetime import datetime

match = re.search(r'\d{4}\d{2}\d{2}.\d{6}', radar_files[0])
match.group()

In [ ]:
match_month = re.search(r'\d{4}\d{2}', radar_files[0])

In [ ]:
match_month.group()

In [ ]:

print(cluster1.logs())

In [ ]:
radar_file_path = radar_files[0]

match_datetime = re.search(r'\d{4}\d{2}\d{2}.\d{6}', radar_file_path)
match_month = re.search(r'\d{4}\d{2}', radar_file_path)
file_datetime = match_datetime.group()
file_month = match_month.group()

In [ ]:
file_month

In [ ]:
file_datetime

In [ ]:
import os

In [ ]:
for file in radar_files:
    match_datetime = re.search(r'\d{4}\d{2}\d{2}.\d{6}', file)
    match_month = re.search(r'\d{4}\d{2}', file)
    file_datetime = match_datetime.group()
    file_month = match_month.group()
    os.rename(file, '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/' +file_month+'/corcsapr2cmacppiM1.c1.'+file_datetime+'.nc')  

In [ ]:
os.rename(radar_file_path, '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/' +file_month+'/corcsapr2cmacppiM1.c1.'+file_datetime+'.nc')

In [ ]:
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppi.c1/' +file_month+'/corcsapr2cmacppiM1.c1.'+file_datetime+'.nc'

In [4]:
import netCDF4
import glob
import numpy as np

In [5]:
radar_directory = '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/*/*'
files = glob.glob(radar_directory, recursive=True)

In [6]:
files

['/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190126.124503.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190126.013003.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190122.104503.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190122.234504.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190124.203003.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190129.131503.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190121.201502.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.20190126.110003.nc',
 '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cmacppiM1.c1/201901/corcsapr2cmacppiM1.c1.2019

In [107]:
def meta_fixer(file):
    try:
        data = netCDF4.Dataset(file, 'r+')
    except OSError:
        print(file + 'failed')
        return
    data.site_id = 'cor'
    data.mentors = 'Bradley Isom, PNNL., Iosif Lindenmaier, PNNL.'
    data.references = 'See CSAPR2 Instrument Handbook'
    data.Conventions = 'CF/Radial instrument_parameters ARM-1.3'
    if 'conventions' in data.ncattrs():
        data.delncattr('conventions')
    data.institution = 'United States Department of Energy - Atmospheric Radiation Measurement (ARM) program'
    data.source = 'Atmospheric Radiation Measurement (ARM) program C-band Scanning ARM Precipitation Radar 2 (CSAPR2)'
    data.attributions = 'This data is collected by the ARM Climate Research facility. Radar system is operated by the radar engineering team radar@arm.gov and the data is processed by the precipitation radar products team. LP code courtesy of Scott Giangrande, BNL.'
    data.comment = 'This is highly experimental and initial data. There are many, known and unknown issues. Please do not use before contacting the Translator responsible scollis@anl.gov'
    data.known_issues = 'False phidp jumps in insect regions. Still uses old Giangrande code.'
    try:
        data.variables['specific_differential_attenuation'].units = 'dB/km'
        data.variables['corrected_velocity'].units = 'm/s'
        data.variables['simulated_velocity'].units = 'm/s'
        data.variables['velocity_texture'].units = 'm/s'
        data.variables['path_integrated_attenuation'].long_name = 'Path Integrated Attenuation'
        data.variables['path_integrated_differential_attenuation'].long_name = 'Path Integrated Differential Attenuation'
        data.variables['rain_rate_A'].comment = 'Rain rate calculated from specific_attenuation, R=51.3*specific_attenuation**0.81, note R=0.0 where norm coherent power < 0.4 or rhohv < 0.8'
    except KeyError:
        data.close()
        return
    #data.variables['ground_clutter'].delncattr('comment')
    if 'standard' in data.variables['ground_clutter'].ncattrs():
        data.variables['ground_clutter'].delncattr('standard_name')
    data.variables['ground_clutter'].flag_values = [0, 1]
    data.variables['ground_clutter'].flag_meanings = 'no_clutter clutter'
    data.variables['ground_clutter'].units = '1'
    data.variables['unfolded_differential_phase'].long_name = 'Unfolded Differential Phase'
    data.variables['filtered_corrected_specific_diff_phase'].long_name = 'Filtered Corrected Specific differential phase (KDP)'
    data.variables['filtered_corrected_differential_phase'].long_name = 'Filtered Corrected Differential Phase'
    data.variables['clutter_masked_velocity'].long_name = 'Radial mean Doppler velocity, positive for motion away from the instrument, clutter removed'
    #data.variables['ground_clutter'].long_name = 'Ground Clutter'
    data.variables['gate_id'].units = '1'
    if 'notes' in data.variables['gate_id'].ncattrs():
        data.variables['gate_id'].delncattr('notes')
    data.variables['gate_id'].flag_values = [0, 1, 2, 3, 4, 5, 6]
    data.variables['gate_id'].flag_meanings = 'multi_trip rain snow no_scatter melting clutter terrain_blockage'
    if 'standard_name' in data.variables['gate_id'].ncattrs():
        data.variables['gate_id'].delncattr('standard_name')
    data.variables['sounding_temperature'].units = 'deg_C'
    data.variables['partial_beam_blockage'].coordinates = 'elevation azimuth range'
    data.variables['cumulative_beam_blockage'].coordinates = 'elevation azimuth range'
    data.variables['height'].units = 'm'
    data.variables['height_over_iso0'].units = 'm'
    data.variables['height_over_iso0'].long_name = 'Height of radar beam over freezing level'
    #data.variables['specific_differential_attenuation'].standard_name = 'specific_differential_attenuation'
    #data.variables['path_integrated_attenuation'].standard_name = 'path_integrated_attenuation'
    #data.variables['path_integrated_differential_attenuation'].standard_name = 'path_integrated_differential_attenuation
    try:
        data.variables['corrected_velocity'].valid_min = np.round(data.variables['corrected_velocity'].valid_min, 4)
        data.variables['corrected_velocity'].valid_max = np.round(data.variables['corrected_velocity'].valid_max, 4)
        data.variables['attenuation_corrected_differential_reflectivity_lag_1'].applied_bias_correction = np.round(
            data.variables['attenuation_corrected_differential_reflectivity_lag_1'].applied_bias_correction, 5)
    except AttributeError:
        data.close()
        return
    data.close()

In [108]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster1 = PBSCluster(processes=6, cores=36, walltime='4:30:00', memory='160GB',
                      name='dask-worker', interface='ib0', queue='arm_high_mem', project='arm',
                      job_extra=['-W group_list=cades-arm'], extra=['--no-dashboard'])
cluster1.scale(4*6)         # Ask for tn workers
client1 = Client(cluster1)  # Connect this local process to remote workers

In [113]:
cluster1

In [115]:
client1

Client Scheduler: tcp://10.23.216.81:33519 Dashboard: http://10.23.216.81:33660/status,Cluster Workers: 12 Cores: 72 Memory: 320.04 GB


In [116]:
the_bag = db.from_sequence(files)
the_function = lambda x: meta_fixer(x)

futures1 = the_bag.map(the_function)

In [117]:
futures1.compute()

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [118]:
!qselect -u zsherman | xargs qdel

tornado.application - ERROR - Uncaught exception GET /status (128.219.164.160)
HTTPServerRequest(protocol='http', host='or-condo-login01.ornl.gov:8787', method='GET', uri='/status', version='HTTP/1.1', remote_ip='128.219.164.160')
Traceback (most recent call last):
  File "/home/zsherman/.conda/envs/cmac_env/lib/python3.7/site-packages/tornado/web.py", line 1703, in _execute
    result = await result
  File "/home/zsherman/.conda/envs/cmac_env/lib/python3.7/site-packages/bokeh/server/views/doc_handler.py", line 58, in get
    template_variables=session.document.template_variables)
  File "/home/zsherman/.conda/envs/cmac_env/lib/python3.7/site-packages/bokeh/embed/server.py", line 237, in server_html_page_for_session
    bundle = bundle_for_objs_and_resources(None, resources)
  File "/home/zsherman/.conda/envs/cmac_env/lib/python3.7/site-packages/bokeh/embed/bundle.py", line 194, in bundle_for_objs_and_resources
    ext = bundle_models(models)
  File "/home/zsherman/.conda/envs/cmac_env

In [42]:
data

NameError: name 'data' is not defined